In [2]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import os
from IPython.display import clear_output
import torch
import numpy as np
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/Shareddrives/Strawberries/Image experiment')

# Function to convert a tensor to an image
def to_img(x):
    return np.moveaxis(x.numpy() * 255, 0, -1).astype(np.uint8)

# Function to get the length-to-width ratio of the largest contour in an image
def getLWR(img):
    # Convert image to grayscale if it has more than one channel
    if img.shape[2] > 1:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Add a constant border to the image
    img = cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=0)
    # Apply binary thresholding
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Invert image if the background is white
    if img[0, 0] == 255:
        img = cv2.bitwise_not(img)
    # Find contours in the image
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get the largest contour by area
    contour = max(contours, key=cv2.contourArea)
    # Get dimensions of the minimum area rectangle enclosing the contour
    w, h = cv2.minAreaRect(contour)[1]
    # Return the length-to-width ratio
    return max(w, h) / min(w, h)

def getShapeIndex(img):
      # Convert image to grayscale if it has more than one channel
    if img.shape[2] > 1:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Add a constant border to the image
    img = cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=0)
    # Apply binary thresholding
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Invert image if the background is white
    if img[0, 0] == 255:
        img = cv2.bitwise_not(img)
    # Find contours in the image
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get the largest contour by area
    contour = max(contours, key=cv2.contourArea)
    # Initialize variables to store the max width and corresponding height
    max_width = 0
    max_height = 0
    point1 = None
    point2 = None
    # Iterate through each pair of points in the contour
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            # Calculate the distance between the two points in both x and y directions
            dx = contour[j][0][0] - contour[i][0][0]
            dy = contour[j][0][1] - contour[i][0][1]
            # If the distance in the x direction (width) is the largest we've seen
            width = abs(dx)
            if width > max_width:
                max_width = width
                max_height = abs(dy)  # The corresponding height (vertical distance)
                point1 = contour[i][0]
                point2 = contour[j][0]
    return max_height / max_width

def getMeanNonBlackColor(img):
  data = img.reshape(-1, img.shape[-1])
  data = data[np.array([np.all(i != [0,0,0]) for i in data])]
  return np.mean(data, axis = 0)

def getRedness(img):
  # image should be rgb
  data = img.reshape(-1, img.shape[-1])
  data = data[np.array([np.all(i != [0,0,0]) for i in data])]
  # redness is distance from (160,20,20)
  distances = np.linalg.norm(data - [160,20,20], axis = 1)
  distances = 255 - np.mean(distances)
  return distances

def binarizeImage(img):
  if(img.shape[2] > 1):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  img = cv2.bitwise_not(img)
  return img

Mounted at /content/drive


In [3]:
geno = torch.load('fullGeno.pt', map_location=torch.device('cpu')).type(torch.float32)
geno.shape

torch.Size([13922, 10322])

In [ ]:
# load images and save genotypes as .csv
images = torch.load('fullImages.pt', map_location=torch.device('cpu')).type(torch.float32)
phenotypesLWR = torch.tensor(np.array([getLWR(to_img(image)) for image in images]))
phenotypesB, phenotypesG, phenotypesR  = torch.tensor(np.array([getMeanNonBlackColor(img = cv2.cvtColor(to_img(image), cv2.COLOR_BGR2RGB)) for image in images])).split(1, dim=1)
phenotypesRedness = torch.tensor(np.array([getRedness(cv2.cvtColor(to_img(image), cv2.COLOR_BGR2RGB)) for image in images]))

phenotypes = pd.DataFrame({
    'LWR': phenotypesLWR.detach().numpy(),
    'B': phenotypesB[:,0].detach().numpy(),
    'G': phenotypesG[:,0].detach().numpy(),
    'R': phenotypesR[:,0].detach().numpy(),
    'Redness': phenotypesRedness.detach().numpy()
  })

phenotypes.to_csv('fullPhenotypes.csv', index=False)
geno = torch.load('fullGeno.pt', map_location=torch.device('cpu')).type(torch.float32)
np.savetxt('fullGeno.csv', geno.detach().numpy(), delimiter=',')

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("rrBLUP")
install.packages("plotly")
require(tidyverse); require(magrittr); require(rrBLUP)
library(data.table)
require(plotly)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/rrBLUP_4.6.3.tar.gz'
Content type 'application/x-gzip' length 17817 bytes (17 KB)
downloaded 17 KB


The downloaded source packages are in
	‘/tmp/RtmpAisUt2/downloaded_packages’
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘lazyeval’, ‘crosstalk’

trying URL 'https://cran.rstudio.com/src/contrib/lazyeval_0.2.2.tar.gz'
Content type 'application/x-gzip' length 83482 bytes (81 KB)
downloaded 81 KB

trying URL 'https://cran.rstudio.com/src/contrib/crosstalk_1.2.1.tar.gz'
Content type 'application/x-gzip' length 297970 bytes (290 KB)
downloaded 290 KB

trying URL 'https://cran.rstudio.com/src/contrib/plotly_4.10.4.tar.gz'
Content type 'application/x-gzip' length 3903133 bytes (3.7 MB)
downloaded 3.7 MB


The downloaded source packages are in
	‘/tmp/RtmpAisUt2/downloaded_packages’
Loading requi

In [ ]:
%%R

takeMean <- function(x, accessions){

  x %>%
    mutate(Accession = accessions) %>%
    group_by(Accession) %>%
    summarise(across(everything(), ~mean(.x, na.rm = T))) %>%
    select(!Accession)

}

correlations <- data.frame(trait = character(), h2 = numeric(), train_r2 = numeric(), test_r2 = numeric())

for (random_seed in 1:50){
  print(random_seed)

  trainKey <- paste0(random_seed, "_trainKey.csv") %>%
    fread(.)

  testKey <- paste0(random_seed, "_testKey.csv") %>%
    fread(.)

  accessions <- trainKey
  accessions[testKey$V1 >= 0] <- testKey$V1[testKey$V1 >= 0]

  genotypes <- "fullGeno.csv" %>%
    fread() %>%
    tibble()

  phenotypes <- "fullPhenotypes.csv" %>%
    fread()

  trainKey %<>% takeMean(., accessions) %>% {.$V1}
  testKey %<>% takeMean(., accessions) %>% {.$V1}
  genotypes %<>% takeMean(., accessions)
  phenotypes %<>% takeMean(., accessions)

  traits <- names(phenotypes)

  trainingSet <- trainKey >= 0
  testingSet <- testKey >= 0
  genotype_matrix <- genotypes %>%
    as.matrix()
  GRM <- A.mat(genotype_matrix)

  rrBLUPpredictedTraits <- phenotypes * 0
  for(trait in traits){
    print(trait)
    estimated_h2 <- c()
    trainCorrelations <- c()
    testCorrelations <- c()

    print(system.time({

      phenotype_vector <- pull(phenotypes, trait)
      phenotype_vector[testingSet] <- NA

      print(system.time({
        fit <- mixed.solve(y = (phenotype_vector - mean(phenotype_vector, na.rm = T)), K = GRM)
      }))
      var_genetic <- fit$Vu
      var_residual <- fit$Ve
      h2 <- var_genetic / (var_genetic + var_residual)
      estimated_h2 <- c(estimated_h2, h2)
      gblups <- (fit$u) + mean(phenotype_vector, na.rm = T)

      rrBLUPpredictedTraits[[trait]] <- as.vector(gblups)

      phenotype_vector <- pull(phenotypes, trait)
      trainCorrelations <- c(trainCorrelations, cor(gblups[trainingSet], phenotype_vector[trainingSet])[1] ^ 2)
      testCorrelations <- c(testCorrelations, cor(gblups[testingSet], phenotype_vector[testingSet])[1] ^ 2)
    }))

    print("h2"); print(estimated_h2)
    print("train r2"); print(trainCorrelations)
    print("test r2"); print(testCorrelations)

    correlations <- rbind(correlations, data.frame(seed = random_seed, trait = trait, h2 = estimated_h2, train_r2 = trainCorrelations, test_r2 = testCorrelations))
    write.csv(correlations, "rrBLUPextractedTraitCorrelations.csv")
  }

  write_csv(rrBLUPpredictedTraits, paste0(random_seed, "_rrBLUPpredictedTraits", ".csv"))

}

[1] 1
[1] "LWR"
   user  system elapsed 
  0.271   0.067   0.313 
   user  system elapsed 
  0.537   0.080   0.600 
[1] "h2"
[1] 0.3081748
[1] "train r2"
[1] 0.4179956
[1] "test r2"
[1] 0.2064658
[1] "B"
   user  system elapsed 
  0.265   0.082   0.183 
   user  system elapsed 
  0.414   0.102   0.343 
[1] "h2"
[1] 0.6965833
[1] "train r2"
[1] 0.7553673
[1] "test r2"
[1] 0.2976125
[1] "G"
   user  system elapsed 
  0.255   0.094   0.185 
   user  system elapsed 
  0.406   0.101   0.347 
[1] "h2"
[1] 0.4055898
[1] "train r2"
[1] 0.5912089
[1] "test r2"
[1] 0.4175517
[1] "R"
   user  system elapsed 
  0.272   0.070   0.177 
   user  system elapsed 
  0.415   0.072   0.321 
[1] "h2"
[1] 0.505059
[1] "train r2"
[1] 0.6379803
[1] "test r2"
[1] 0.2868551
[1] "Redness"
   user  system elapsed 
  0.268   0.093   0.206 
   user  system elapsed 
  0.411   0.100   0.351 
[1] "h2"
[1] 0.4212464
[1] "train r2"
[1] 0.60031
[1] "test r2"
[1] 0.4106292
[1] 2
[1] "LWR"
   user  system elapsed 
  0.338 

|--------------------------------------------------|
|==================================================|
|--------------------------------------------------|
|==================================================|
|--------------------------------------------------|
|==================================================|
|--------------------------------------------------|
|==================================================|
|--------------------------------------------------|
|==================================================|
|--------------------------------------------------|
|==================================================|
|--------------------------------------------------|
|==================================================|
|--------------------------------------------------|
|==================================================|
|--------------------------------------------------|
|==================================================|
|---------------------------------------------